In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from transformers import BertTokenizerFast

import glob
import pandas as pd
from sklearn.utils import shuffle
from tqdm.notebook import tqdm

fpath = './data/tokenizer_model'
tokenizer = BertTokenizerFast.from_pretrained(fpath,
                                              strip_accents=False,
                                              lowercase=False)

vocab_dim     = len(tokenizer.vocab)
seq_len       = 256
embedding_dim = 512
device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size    = 256

In [2]:
# https://github.com/GyuminJack/torchstudy/blob/main/06Jun/NER/src/data.py

import linecache
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from torch.nn.utils.rnn import pad_sequence

def load_tokenizer(tokenizer_path):
    loaded_tokenizer = BertTokenizerFast.from_pretrained(tokenizer_path, strip_accents=False, lowercase=False)  # Must be False if cased model  # 로드
    return loaded_tokenizer

class KlueDataset_NER(Dataset):
    def __init__(self, vocab_txt_path, txt_path, *args, **kwargs):
        self.tokenizer = load_tokenizer(vocab_txt_path)
        self.max_seq_len = 256
        self.txt_path = txt_path
        
        self.cls_token_id  = self.tokenizer.cls_token_id
        self.sep_token_id  = self.tokenizer.sep_token_id
        self.pad_token_id  = self.tokenizer.pad_token_id
        
        self.bio_dict = {
                        '[PAD]' : 0,
                        'B-DT': 1,
                        'B-LC': 2,
                        'B-OG': 3,
                        'B-PS': 4,
                        'B-QT': 5,
                        'B-TI': 6,
                        'I-DT': 7,
                        'I-LC': 8,
                        'I-OG': 9,
                        'I-PS': 10,
                        'I-QT': 11,
                        'I-TI': 12,
                        'O': 13
                        }
        self.reverse_bio_dict = {v:k for k, v in self.bio_dict.items()}
        with open(self.txt_path, "r") as f:
            self._total_data = len(f.readlines())

    def __len__(self):
        return self._total_data

    def __getitem__(self, idx):
        raw_ko = linecache.getline(self.txt_path, idx + 1).strip()
        text, bio_string = raw_ko.split("\t")
        bio_tensor = [self.bio_dict[i] for i in bio_string.split(",")]
        
        sent = self.tokenizer.encode(text)[1:-1]
        pad_length = self.max_seq_len - 2 - len(sent)
        
        train = torch.tensor([self.cls_token_id] + sent + [self.sep_token_id] + [self.pad_token_id] * pad_length).long().contiguous()
        target = torch.tensor([self.cls_token_id] + bio_tensor + [self.sep_token_id] + [self.pad_token_id] * pad_length).long().contiguous()
        
        segment_embedding = torch.zeros(target.size(0)).long()
        
        return train, target, segment_embedding

In [3]:
# https://inhyeokyoo.github.io/project/nlp/bert-issue/

import torch
import torch.nn as nn

class BERT(nn.Module):
    def __init__(self, vocab_dim, seq_len, embedding_dim, pad_token_id):
        super(BERT, self).__init__()
        self.pad_token_id  = pad_token_id
        self.nhead         = 8
        self.embedding     = BERTEmbedding(vocab_dim, seq_len, embedding_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=self.nhead, batch_first=True)
        self.encoder_block = nn.TransformerEncoder(self.encoder_layer, num_layers=6)
        
        
    def forward(self, data, segment_embedding):
        pad_mask  = BERT.get_attn_pad_mask(data, data, self.pad_token_id).repeat(self.nhead, 1, 1)
        embedding = self.embedding(data, segment_embedding)
        output    = self.encoder_block(embedding, pad_mask) 
        
        return output
    
    @staticmethod
    def get_attn_pad_mask(seq_q, seq_k, i_pad):
        batch_size, len_q = seq_q.size()
        batch_size, len_k = seq_k.size()
        pad_attn_mask = seq_k.data.eq(i_pad)
        pad_attn_mask= pad_attn_mask.unsqueeze(1).expand(batch_size, len_q, len_k)
        
        return pad_attn_mask

In [4]:
class BERTEmbedding(nn.Module):
    def __init__(self, vocab_dim, seq_len, embedding_dim, dropout_rate=0.1, device=device):
        super(BERTEmbedding, self).__init__()
        self.seq_len       = seq_len
        self.vocab_dim     = vocab_dim
        self.embedding_dim = embedding_dim
        self.dropout_rate  = dropout_rate
        
        # vocab --> embedding
        self.token_embedding      = nn.Embedding(self.vocab_dim, self.embedding_dim) 
        self.token_dropout        = nn.Dropout(self.dropout_rate)    
        
        # seq len --> embedding
        self.positional_embedding = nn.Embedding(self.seq_len, self.embedding_dim)
        self.positional_dropout   = nn.Dropout(self.dropout_rate) 
        
        # segment (0, 1) --> embedding
        self.segment_embedding    = nn.Embedding(2, self.embedding_dim)
        self.segment_dropout      = nn.Dropout(self.dropout_rate) 
        
        
    def forward(self, data, segment_embedding):
        token_embedding      = self.token_embedding(data)
        token_embedding      = self.token_dropout(token_embedding)
        
        positional_encoding  = torch.arange(start=0, end=self.seq_len, step=1).long()
        # data의 device 정보 가져와서 처리
        positional_encoding  = positional_encoding.unsqueeze(0).expand(data.size()).to(device)
        positional_embedding = self.positional_embedding(positional_encoding)
        positional_embedding = self.positional_dropout(positional_embedding)
        
        segment_embedding    = self.segment_embedding(segment_embedding)
        segment_embedding    = self.segment_dropout(segment_embedding)
        
        return token_embedding + positional_embedding + segment_embedding

In [5]:
from torchcrf import CRF

class BertNER(nn.Module):
    def __init__(self, bert, bert_hidden_size, num_classes):
        super().__init__()
        self.bert = bert
        self.fc = nn.Linear(bert_hidden_size, num_classes)
        self.dropout = nn.Dropout(0.1)
        self.crf = CRF(num_tags=num_classes, batch_first=True)

    def forward(self, source, target, segment_embedding):
        source = self.bert(source, segment_embedding)
        last_encoder_output = self.dropout(source)
        emissions = self.fc(last_encoder_output)
        log_likelihood, output = self.crf(emissions, target), self.crf.decode(emissions)
        
        return log_likelihood, torch.tensor(output)

In [6]:
def train(model, iterator, optimizer, device, clip=1):
    model.train()
    
    epoch_loss = 0
    epoch_acc  = 0
    
    for source, target, segment_embedding in tqdm(iterator, total=len(iterator)):
        optimizer.zero_grad()
        
        source = source.to(device)
        target = target.to(device)
        segment_embedding = segment_embedding.to(device)

        log_likelihood, output = model(source, target, segment_embedding) 
        
        loss = -1 * log_likelihood
        
        loss.backward()
                
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        target = target.to("cpu")
        output = output.to('cpu')

        non_zero_index =non_zero_index = (target != 0)
        target_non_zero = target[non_zero_index]
        output_non_zero = output[non_zero_index]

        acc = ((target_non_zero==output_non_zero).float().sum() / len(target_non_zero))

        epoch_loss += loss.item()
        epoch_acc  += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


@torch.no_grad()
def evaluate(model, iterator, optimizer, device, clip=1):
    model.eval()
    
    epoch_loss = 0
    epoch_acc  = 0
    
    for source, target, segment_embedding in iterator:
        optimizer.zero_grad()
        
        source = source.to(device)
        target = target.to(device)
        segment_embedding = segment_embedding.to(device)

        log_likelihood, output = model(source, target, segment_embedding) 

        loss = -1 * log_likelihood
        
        target = target.to("cpu")
        output = output.to('cpu')

        non_zero_index =non_zero_index = (target != 0)
        target_non_zero = target[non_zero_index]
        output_non_zero = output[non_zero_index]

        acc = ((target_non_zero==output_non_zero).float().sum() / len(target_non_zero))
        epoch_loss += loss.item()
        epoch_acc  += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [7]:
vocab_txt_path = "./data/tokenizer_model"

train_path = "./data/klue_ner_processed.train"
train_dataset = KlueDataset_NER(vocab_txt_path, train_path)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_path = "./data/klue_ner_processed.dev"
valid_dataset = KlueDataset_NER(vocab_txt_path, valid_path)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size)

checkpoint = torch.load('./weights/BERT_LM_best.pt')
bert = BERT(vocab_dim=vocab_dim, seq_len=seq_len, embedding_dim=embedding_dim, pad_token_id=0).to(device)
bert.load_state_dict(checkpoint['bert'])

ner_head = BertNER(bert, embedding_dim, len(train_dataset.bio_dict)).to(device)

optimizer = torch.optim.AdamW(ner_head.parameters(), lr = 0.0001, weight_decay = 0.01)
scheduler = ReduceLROnPlateau(optimizer, 'min')

In [ ]:
start_epoch = 0
N_EPOCHS  = 1000
PAITIENCE = 30

n_paitience = 0
best_valid_loss = float('inf')
optimizer.zero_grad()
optimizer.step()

for epoch in range(start_epoch, N_EPOCHS):
    train_loss, train_acc = train(ner_head, train_data_loader, optimizer, device)
    valid_loss, valid_acc = evaluate(ner_head, valid_data_loader, optimizer, device)
    scheduler.step(valid_loss)

    print(f'Epoch: {epoch + 1:04}')
    print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}')
    print(f'Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}')

    with open("output/log_ner.txt", "a") as f:
        f.write("epoch: {0:04d} train loss: {1:.4f}, train acc: {2:.4f}, valid loss: {3:.4f}, valid acc: {4:.4f} \n".format(epoch, train_loss, train_acc, valid_loss, valid_acc))


    if n_paitience < PAITIENCE:
        if best_valid_loss > valid_loss:
            best_valid_loss = valid_loss
            torch.save(
                {
                'ner_head' : ner_head.state_dict(),
                'optimizer': optimizer.state_dict()
                }, 'weights/BERT_ner_best.pt'
            )
            n_paitience = 0
        elif best_valid_loss <= valid_loss:
            n_paitience += 1
    else:
        print("Early stop!")
        checkpoint = torch.load('weights/BERT_ner_best.pt')
        ner_head.load_state_dict(checkpoint['ner_head'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        break

  0%|          | 0/83 [00:00<?, ?it/s]

/home/ubuntu/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/aten/src/ATen/native/TensorCompare.cpp:255.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch: 0001
Train Loss: 10316.1697 | Train Acc: 0.8016
Valid Loss: 2992.3516 | Valid Acc: 0.8855


  0%|          | 0/83 [00:00<?, ?it/s]

Epoch: 0002
Train Loss: 2398.7176 | Train Acc: 0.9062
Valid Loss: 2395.5992 | Valid Acc: 0.9075


...

  0%|          | 0/83 [00:00<?, ?it/s]

Epoch: 0017
Train Loss: 541.7198 | Train Acc: 0.9763
Valid Loss: 1870.0714 | Valid Acc: 0.9399


  0%|          | 0/83 [00:00<?, ?it/s]

Epoch: 0018
Train Loss: 508.3576 | Train Acc: 0.9776
Valid Loss: 1847.8342 | Valid Acc: 0.9413


  0%|          | 0/83 [00:00<?, ?it/s]